# Title
[]()

# Create classes

In [5]:
from with_sql_session import remote_sql_session
import secrets
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid


# def bulk_save_objects(new_rows, engine=None):
#     @remote_sql_session
#     def save_objects(session):
#         session.bulk_save_objects(new_rows)
#         session.commit()

#     return save_objects


Base = declarative_base()

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True)
    title = Column(String(255))
    publication = Column(String(100))
    text_body = Column(Text)
    uuid = Column(UUID)
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

new_rows = [
    Article(
        id="1", title="Test Article", 
        publication="Test Journal", text_body="This is a test text", uuid=str(uuid.uuid4())
        )
    ]

# save_objects = bulk_save_objects(new_rows)
# save_objects()

# Get table

## 1.1 using .query

In [8]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
import pandas as pd


@remote_sql_session
def get_table(session, table='articles', filter=None):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.query(Article).all()
    # df = pd.DataFrame(q)
    # # df = pd.DataFrame(q.fetchall())
    # return df
    return q


table = get_table()

In [12]:
vars(table[0])

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1cb95f9e410>,
 'id': 1,
 'text_body': 'This is a test text',
 'title': 'Test Article',
 'uuid': 'a678c60d-f4a6-4727-bbaa-cc2cc958f52d',
 'publication': 'Test Journal'}

## 1.2 using .execute

In [13]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
import pandas as pd


@remote_sql_session
def get_table(session, query='SELECT *', table='articles'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df


get_table()

,id,title,publication,text_body,uuid
0,1,Test Article,Test Journal,This is a test text,a678c60d-f4a6-4727-bbaa-cc2cc958f52d


# Update tables

In [3]:

from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True)
    title = Column(String(255))
    publication = Column(String(100))
    text_body = Column(Text)
    uuid = Column(UUID)
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

In [8]:
get_table()

NameError: name 'get_table' is not defined

## 2.0

In [10]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
import pandas as pd

def bulk_save_objects(new_rows, engine=None):
    @remote_sql_session
    def save_objects(session):
        session.bulk_save_objects(new_rows)
        session.commit()

    return save_objects


@remote_sql_session
def get_table(session, query='SELECT *', table='articles'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

new_rows = [
    Publications(
        id=1, name="PLOS One"
        )
    ]

# save_objects = bulk_save_objects(new_rows)
# save_objects()

In [12]:
get_table(table='publications')

,id,name
0,1,PLOS One


## 2.1

In [14]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
import pandas as pd
from sqlalchemy.dialects.postgresql import insert

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        statement = insert(Publications).values(new_rows)
        session.execute(statement)
        session.commit()
        print('Rows inserted:')
        print([f"\t{row['name']}" for row in new_rows])

    return insert_rows

values = [
    {'id': 2, 'name': 'BMJ Open'},
]
bulk_insert(values)()

get_table()

Rows inserted:
['\tBMJ Open']


,id,name
0,1,PLOS One
1,2,BMJ Open


# 3 Update tables

In [15]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
# from sqlalchemy.dialects.postgresql import UUID
# import uuid
import pandas as pd
from sqlalchemy.dialects.postgresql import insert


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication = Column(String(100))
    text_body = Column(Text)
    uuid = Column(UUID)
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        statement = insert(Publications).values(new_rows)
        session.execute(statement)
        session.commit()
        print('Rows inserted:')
        print([f"\t{row['name']}" for row in new_rows])

    return insert_rows

values = [
    {'name': 'PLOS Medicine'},
]
bulk_insert(values)()

get_table()

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "id" of relation "publications" violates not-null constraint
DETAIL:  Failing row contains (null, PLOS Medicine).

[SQL: INSERT INTO publications (name) VALUES (%(name_m0)s)]
[parameters: {'name_m0': 'PLOS Medicine'}]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

## 3.1 Able to insert rows using .add

In [2]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
from sqlalchemy.dialects.postgresql import insert


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication = Column(String(100))
    text_body = Column(Text)
    uuid = Column(UUID)
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            # new_row = Publications(
            #     name=row['name']
            # )
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

values = [
    Publications(name='PLOS Medicine'),
]
bulk_insert(values)()

get_table()

Rows inserted:


,id,name
0,1,PLOS Medicine


In [3]:
values = [
    Publications(name='PLOS One'),
]
bulk_insert(values)()

get_table()

Rows inserted:


,id,name
0,1,PLOS Medicine
1,2,PLOS One


# Iteration 4: Relate tables to each other.

In [1]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    articles = relationship('Article', back_populates='publication')

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication = Column(String(100))
    text_body = Column(Text)
    uuid = Column(UUID)
    publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            # new_row = Publications(
            #     name=row['name']
            # )
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

values = [
    Publications(name='PLOS Medicine'),
]
bulk_insert(values)()

get_table()


NoForeignKeysError: Could not determine join condition between parent/child tables on relationship Publications.articles - there are no foreign keys linking these tables.  Ensure that referencing columns are associated with a ForeignKey or ForeignKeyConstraint, or specify a 'primaryjoin' expression.

## 4.1

In [7]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")

from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import relationship


Base = declarative_base()

class Publications(Base):
    __tablename__ = 'publications'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    # articles = relationship('Article', back_populates='publication')

class Article(Base):
    __tablename__ = 'articles'
    id = Column(Integer, primary_key=True, server_default='DEFAULT', autoincrement=True)
    title = Column(String(255))
    publication = Column(String(100))
    text_body = Column(Text)
    uuid = Column(UUID)
    # publication = relationship('Publications', back_populates='articles')
    
class Summaries(Base):
    __tablename__ = 'initial_summaries'
    id = Column(Integer, primary_key=True)
    summary_body = Column(String(1000))
    article_uuid = Column(UUID)

@remote_sql_session
def get_table(session, query='SELECT *', table='publications'):
    """
    Return a database table as a pandas dataframe.
    """
    q = session.execute(f'{query} from {table}')
    df = pd.DataFrame(q.fetchall())
    return df

def bulk_insert(new_rows, engine=None):
    @remote_sql_session
    def insert_rows(session):
        print('Rows inserted:')
        for row in new_rows:
            # new_row = Publications(
            #     name=row['name']
            # )
            session.add(row)
            session.commit()
            # print([f"\t{row['name']}"])
            # print([f"\t{row}"])

    return insert_rows

values = [
    Publications(
        name='Journal of Applied Physiology',
        ),
]
# article_list = [
#     Article(
#         title='Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women',
#         ),
# ]
bulk_insert(values)()
get_table()
# get_table(table='articles')


Rows inserted:


,id,name
0,1,PLOS Medicine
1,2,PLOS One
2,3,Journal of Applied Physiology


# *End of Page*